# RAG (Retrieval Augmented Generation)

- 자체 문서를 사용하여 응답을 생성 (RAG 프로세스에 따라 진행해보기!)

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### Phase 1: Indexing

##### [1] Load
- 데이터 로드
- Document Loader 시용

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('./snow-white.pdf')
documents = loader.load()

documents

[Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'moddate': '2023-09-12T11:20:24+09:00', 'source': './snow-white.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='백설공주\n옛날 어느 왕국에 공주님이 태어났어요.\n“어쩜 이렇게 어여쁠까? 살결이 눈처럼 하얗구나. 백\n설공주라고 불러야겠다.”\n왕과 왕비는 갓 태어난 딸을 보며 기뻐했어요.\n하지만 기쁨도 잠시, 왕비는 곧 세상을 떠나고 말았어\n요.'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'moddate': '2023-09-12T11:20:24+09:00', 'source': './snow-white.pdf', 'total_pages': 6, 'page': 1, 'page_label': '2'}, page_content='왕은 아름다운 새 왕비를 맞았어요.\n그런데 새 왕비는 자기보다 아름다운 사람을 두고 보\n지 못했어요.\n왕비는 진실만을 말하는 요술 거울에게 늘 이렇게 물\n었어요.\n“거울아, 거울아. 이 세상에서 누가 가장 아름답니?”\n“이 세상에서 가장 아름다운 사람은 왕비님입니다.”\n그 대답을 들어야만 차가운 왕비 얼굴에 미소가 번졌\n지요.\n시간이 흘러 백설공

In [3]:
for doc in documents:
    doc.page_content = doc.page_content.replace('\n', ' ')
    
documents

[Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'moddate': '2023-09-12T11:20:24+09:00', 'source': './snow-white.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='백설공주 옛날 어느 왕국에 공주님이 태어났어요. “어쩜 이렇게 어여쁠까? 살결이 눈처럼 하얗구나. 백 설공주라고 불러야겠다.” 왕과 왕비는 갓 태어난 딸을 보며 기뻐했어요. 하지만 기쁨도 잠시, 왕비는 곧 세상을 떠나고 말았어 요.'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'moddate': '2023-09-12T11:20:24+09:00', 'source': './snow-white.pdf', 'total_pages': 6, 'page': 1, 'page_label': '2'}, page_content='왕은 아름다운 새 왕비를 맞았어요. 그런데 새 왕비는 자기보다 아름다운 사람을 두고 보 지 못했어요. 왕비는 진실만을 말하는 요술 거울에게 늘 이렇게 물 었어요. “거울아, 거울아. 이 세상에서 누가 가장 아름답니?” “이 세상에서 가장 아름다운 사람은 왕비님입니다.” 그 대답을 들어야만 차가운 왕비 얼굴에 미소가 번졌 지요. 시간이 흘러 백설공주는 어여쁜 소녀가 되었어요

##### [2] Split

- Text Splitter: 큰 문서 -> 작은 chunk 분할
    - 데이터를 인덱싱하거나 모델에 전달할 때 유용
    - chunk가 커도 문제 (그런데 chunk가 너무 작아도 문제)


In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text = '대한민국의 역사는 매우 길고 다양하다. 고조선부터 시작해서 삼국시대, 고려, 조선, 현대에 이르기까지 수많은 사건과 인물이 존재한다.'

splitter = RecursiveCharacterTextSplitter(
    chunk_size=30,
    chunk_overlap=10
)

splitter.split_text(text)

['대한민국의 역사는 매우 길고 다양하다. 고조선부터',
 '고조선부터 시작해서 삼국시대, 고려, 조선, 현대에',
 '조선, 현대에 이르기까지 수많은 사건과 인물이',
 '사건과 인물이 존재한다.']

In [10]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20
)

docs = splitter.split_documents(documents)
docs

[Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'moddate': '2023-09-12T11:20:24+09:00', 'source': './snow-white.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='백설공주 옛날 어느 왕국에 공주님이 태어났어요. “어쩜 이렇게 어여쁠까? 살결이 눈처럼 하얗구나. 백 설공주라고 불러야겠다.” 왕과 왕비는 갓 태어난 딸을 보며 기뻐했어요. 하지만'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'moddate': '2023-09-12T11:20:24+09:00', 'source': './snow-white.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='딸을 보며 기뻐했어요. 하지만 기쁨도 잠시, 왕비는 곧 세상을 떠나고 말았어 요.'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author':

##### [3] Store

- 분할된 chunk를 저장하고 indexing 할 저장소 필요
    - VectorStore(VectoreDB), Embedding Model 사용

In [11]:
# 임베딩 모델
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [ ]:
#!pip install langchain-chroma

  Using cached chromadb-1.0.20-cp39-abi3-win_amd64.whl.metadata (7.4 kB)
  Using cached build-1.3.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached uvicorn-0.35.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached opentelemetry_api-1.36.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.36.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_sdk-1.36.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached PyPika-0.48.9.tar.gz (67 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached importlib_resources-6.5.2-py3-none-

In [13]:
from langchain_chroma.vectorstores import Chroma

vector_store = Chroma.from_documents(docs, embedding_model)

- 간단한 유사도 기반 검색

In [14]:
query = '백설공주와 왕비 중에 누가 더 아름답나요?'

In [17]:
# 1. vector store를 통한 직접 검색
results = vector_store.similarity_search_with_score(query, k=3)
results

[(Document(id='1efe939d-31b4-4a80-9f0d-cf53da898e10', metadata={'source': './snow-white.pdf', 'page_label': '2', 'creationdate': '2023-09-12T11:20:24+09:00', 'total_pages': 6, 'title': 'PowerPoint 프레젠테이션', 'producer': 'Microsoft® PowerPoint® 2013', 'moddate': '2023-09-12T11:20:24+09:00', 'author': 'PC', 'creator': 'Microsoft® PowerPoint® 2013', 'page': 1}, page_content='거울아. 이 세상에서 누가 가장 아름답니?” “왕비님도 아름답지만 백설공주가 더 아름답습니다.” 화가 난 왕비는 사냥꾼을 불렀어요. 왕비는 사냥꾼에게 백설공주를 죽이라고'),
  0.8401364088058472),
 (Document(id='0139e414-891a-4ed7-a33f-b2076569bfa3', metadata={'page_label': '3', 'producer': 'Microsoft® PowerPoint® 2013', 'moddate': '2023-09-12T11:20:24+09:00', 'creator': 'Microsoft® PowerPoint® 2013', 'page': 2, 'author': 'PC', 'total_pages': 6, 'creationdate': '2023-09-12T11:20:24+09:00', 'source': './snow-white.pdf', 'title': 'PowerPoint 프레젠테이션'}, page_content='왕비는 다시 요술 거울에게 누가 가장 아름다운 지 물었어요. “왕비님도 아름답지만 백설공주님이 천배는 더 아름답습니다.” “사냥꾼이 날 속였구나. 내가 직접 해치우겠어!”'),
  0.8603643774986267),
 (Document(

In [16]:
# 2. Retriever를 활용한 검색
retriever = vector_store.as_retriever(
    search_type='similarity',
    search_kwargs={'k':3}
)

retriever_result = retriever.batch([query])
retriever_result

[[Document(id='1efe939d-31b4-4a80-9f0d-cf53da898e10', metadata={'moddate': '2023-09-12T11:20:24+09:00', 'source': './snow-white.pdf', 'creator': 'Microsoft® PowerPoint® 2013', 'producer': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'total_pages': 6, 'title': 'PowerPoint 프레젠테이션', 'page': 1, 'page_label': '2', 'author': 'PC'}, page_content='거울아. 이 세상에서 누가 가장 아름답니?” “왕비님도 아름답지만 백설공주가 더 아름답습니다.” 화가 난 왕비는 사냥꾼을 불렀어요. 왕비는 사냥꾼에게 백설공주를 죽이라고'),
  Document(id='0139e414-891a-4ed7-a33f-b2076569bfa3', metadata={'page': 2, 'author': 'PC', 'total_pages': 6, 'source': './snow-white.pdf', 'producer': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'moddate': '2023-09-12T11:20:24+09:00', 'creator': 'Microsoft® PowerPoint® 2013', 'page_label': '3'}, page_content='왕비는 다시 요술 거울에게 누가 가장 아름다운 지 물었어요. “왕비님도 아름답지만 백설공주님이 천배는 더 아름답습니다.” “사냥꾼이 날 속였구나. 내가 직접 해치우겠어!”'),
  Document(id='466b663d-df83-4d04-b0e5-20048ff0308c', met

### Phase 2: Retrieval and Generation 

- 사용자 입력(질의)가 주어지면

1. Retriever를 사용하여 저장소에서 관련된 chunk 검색
2. 질문과 검색된 데이터를 포함하는 프롬프트를 통해
3. LLM Model이 답변을 생성

##### (1) PrompltTemplate 생성

- 사용자 질의 + 검색된 문서

In [19]:
from langchain_core.prompts import ChatPromptTemplate

prompt_tpl = ChatPromptTemplate([
    ('system', '당신은 어린 아이에게 꿈과 희망을 심어주는 유치원 교사입니다. 질문하는 아이에게 최대한 호응해주며 context 기반으로만 답변해 주세요.'),
    ('user', '''어린이의 질문에 context만을 이용해 답변하세요.
        context에서 확인할 수 없는 질문이라면 모른다고 답변해야 합니다.
        최종 응답에는 참조한 context에 대한 정보를 추가해 주세요.
     
        질문: {query}
        context: {context}
        
        <<최종 응답 형식>>
        답변: 
        참조 문서:
        - <<source>> (page: <<page>>): <<page_content>>
     ''')
])

In [20]:
prompt_tpl.invoke({'query':query, 'context': retriever_result})

ChatPromptValue(messages=[SystemMessage(content='당신은 어린 아이에게 꿈과 희망을 심어주는 유치원 교사입니다. 질문하는 아이에게 최대한 호응해주며 context 기반으로만 답변해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content="어린이의 질문에 context만을 이용해 답변하세요.\n        context에서 확인할 수 없는 질문이라면 모른다고 답변해야 합니다.\n        최종 응답에는 참조한 context에 대한 정보를 추가해 주세요.\n\n        질문: 백설공주와 왕비 중에 누가 더 아름답나요?\n        context: [[Document(id='1efe939d-31b4-4a80-9f0d-cf53da898e10', metadata={'moddate': '2023-09-12T11:20:24+09:00', 'source': './snow-white.pdf', 'creator': 'Microsoft® PowerPoint® 2013', 'producer': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'total_pages': 6, 'title': 'PowerPoint 프레젠테이션', 'page': 1, 'page_label': '2', 'author': 'PC'}, page_content='거울아. 이 세상에서 누가 가장 아름답니?” “왕비님도 아름답지만 백설공주가 더 아름답습니다.” 화가 난 왕비는 사냥꾼을 불렀어요. 왕비는 사냥꾼에게 백설공주를 죽이라고'), Document(id='0139e414-891a-4ed7-a33f-b2076569bfa3', metadata={'page': 2, 'author': 'PC', 'total_pages': 6, 'source': './snow-white.pdf', 'producer': 'M

##### (2) LLM 

In [21]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.5
)

##### (3) Chain 생성

In [22]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt_tpl | model | StrOutputParser()

##### 사용

In [25]:
query = '왕비가 백설공주에게 먹인 것은 무엇인가요?'
retrievals = retriever.batch([query])
#context_text = '\n'.join(doc.page_content for doc in retrievals[0])

response = chain.invoke({'query':query, 'context':retrievals})
#response = chain.invoke({'query':query, 'context':context_text})

In [26]:
print(response)

답변: 왕비가 백설공주에게 먹인 것은 독이 발라진 사과예요. 왕비는 사과를 먹음직스럽게 꾸며서 백설공주에게 주었답니다.

참조 문서:
- ./snow-white.pdf (page: 4): "왕비는 먹음직스럽게 생긴 사과를 골라 독을 발랐어요."


---

In [27]:
# RetrievalQA  사용
from langchain.chains import RetrievalQA

retrieval_qa = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type = 'stuff'
)
response = retrieval_qa.invoke('난쟁이는 몇 명인가요')

In [28]:
print(response)

{'query': '난쟁이는 몇 명인가요', 'result': '일곱 명입니다.'}
